In [1]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, Tool
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import Graph, StateGraph, START, END
from langgraph.graph.message import add_messages

from typing_extensions import TypedDict
from typing import Dict, List, Optional, Tuple, Annotated

# from tools import generate_few_shot

### Native calls to LLMs.
#### The output format is dependent on the LLM chosen. 

In [4]:
import json

with open("../data/drift_monitoring_cot_examples.json") as f:
    cot = json.load(f)
print(cot)

[{'question': 'For the prompt:What is the weather today?,Consider a binary text classification problem. which class does the prompt belong? Strictly choose one of the two classes. The options are: (1)  Healthcare insurance or (2) other', 'answer': 'other', 'explanation': 'Other since weather has nothing to do with healthcare insurance'}, {'question': 'For the prompt:What is the waste generated at Texas plant last month?,Consider a binary text classification problem. which class does the prompt belong? Strictly choose one of the two classes. The options are: (1)  Waste management or (2) other', 'answer': 'Waste management', 'explanation': 'Waste management since waste information is being retrieved for last month. '}, {'question': 'For the prompt:Can I claim insurance on a direct appointment,Consider a binary text classification problem. which class does the prompt belong? Strictly choose one of the two classes. The options are: (1)  Healthcare insurance or (2) other', 'answer': 'Health

In [6]:
usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

### Function calling with LLMs.
#### The output format should be independent on the LLM chosen. 

In [5]:
from langchain_core.messages import AIMessage
from langgraph.graph import END, StateGraph
from typing import TypedDict
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [7]:
from langchain_ollama import ChatOllama

ollama = ChatOllama(
    model="llama3.2",
    temperature=0.0,
    num_predict=256,
    # other params ...
)

In [8]:
from pydantic import BaseModel, Field


class AITaskFormat(BaseModel):
    """AI Task Format."""

    answer: str = Field(description="AI Tasks")
    question: str = Field(description="The question on prompt relevance")
    explanation: int = Field(
        description="Explanations for why the prompt is relevant or not relevant"
    )

In [9]:
# Reference: https://blog.gopenai.com/zeroshot-fewshot-and-prompt-chaining-using-langchain-4259d700d67f
class FewShot:
    def __init__(self):
        self.examples = []
        self.prefix = f"""
        I want you to play the role of a compliance officer and answer the question
        Return the question, answer and explanation in a json format where question, answer and explanation are keys of the json exactly as shown in the examples.
        you should answer the question followed by an explanation on how that answer was generated. 
        """

    def set_examples(self, examples):
        self.examples = examples
        return self.examples

    def get_examples(self):
        return self.examples

    def get_example_template(self):
        template = """
        Question: {question}
        Answer: {answer}
        Explanation: {explanation}
        """
        example_variables = ["question", "answer", "explanation"]
        return template, example_variables

    def get_prefix(self):
        return f"""
        I want you to play the role of a compliance officer and answer the question
        Return the question, answer and explanation in a json format where question, answer and explanation are keys of the json exactly as shown in the examples.
        you should answer the question followed by an explanation on how that answer was generated. 
        """

    def get_suffix(self):
        return """
        Question: {question}
        """

In [10]:
# Reference: https://blog.gopenai.com/zeroshot-fewshot-and-prompt-chaining-using-langchain-4259d700d67f

import os
from langchain_core.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


class FewShotUtility:

    def __init__(
        self,
        examples,
        prefix,
        suffix,
        input_variables,
        example_template,
        example_variables,
    ):
        self.examples = examples
        self.prefix = prefix
        self.suffix = suffix
        self.input_variables = input_variables
        self.example_template = example_template
        self.example_variables = example_variables

    def get_prompt(self, question):
        prompt_template = FewShotPromptTemplate(
            examples=self.examples,
            example_prompt=self.get_prompt_template(),
            prefix=self.prefix,
            suffix=self.suffix,
            input_variables=self.input_variables,
        )
        prompt = prompt_template.format(question=question)
        return prompt

    def get_prompt_template(self):
        example_prompt = PromptTemplate(
            input_variables=self.example_variables, template=self.example_template
        )
        return example_prompt

    def user_drift_detection(self, prompt):
        prompt_template = ChatPromptTemplate.from_template(prompt)
        message = prompt_template.format_messages()
        structured_llm = ollama.with_structured_output(
            AITaskFormat, method="function_calling"
        )
        response = structured_llm.invoke(message)
        return response

In [11]:
from typing import Optional

from typing_extensions import Annotated, TypedDict


# TypedDict
class AITaskFormat(TypedDict):

    question: Annotated[str, ..., "The question and use case of the prompt"]
    answer: Annotated[str, ..., "Domain for the use case"]
    explanation: Annotated[str, None, "Explanations for why the domain was selected"]

In [14]:
fewshot = FewShot()
fewshot.set_examples(examples=cot)

[{'question': 'For the prompt:What is the weather today?,Consider a binary text classification problem. which class does the prompt belong? Strictly choose one of the two classes. The options are: (1)  Healthcare insurance or (2) other',
  'answer': 'other',
  'explanation': 'Other since weather has nothing to do with healthcare insurance'},
 {'question': 'For the prompt:What is the waste generated at Texas plant last month?,Consider a binary text classification problem. which class does the prompt belong? Strictly choose one of the two classes. The options are: (1)  Waste management or (2) other',
  'answer': 'Waste management',
  'explanation': 'Waste management since waste information is being retrieved for last month. '},
 {'question': 'For the prompt:Can I claim insurance on a direct appointment,Consider a binary text classification problem. which class does the prompt belong? Strictly choose one of the two classes. The options are: (1)  Healthcare insurance or (2) other',
  'answ

In [13]:
prompts = [
    "Create a personalized response for a customer who inquired about returning a defective product. The customer's name is 'Emily', and she has made two previous purchases from our company. Please include a summary of the return policy, a recommendation for similar products, and a suggestion for how Emily can proceed with her return",
    "Generate a summary of claims for a customer who inquired about the 'Eco-Friendly' label on our company's cleaning products. The customer is concerned about the environmental impact of the products and wants to know more about how we ensure sustainability in our manufacturing process",
    "Create a character profile for a fictional character for my personal venture",
    "Translate French to English.",
    "I'm experiencing abdominal pain and vomiting after eating a meal. What could be causing this?",
    "I'm looking for information on natural remedies for stress relief. Can you recommend any supplements or herbs?",
    "Recommend a solution for a customer who inquired about replacing a faulty part on their product. The customer has tried troubleshooting but was unable to resolve the issue, and they're concerned that the replacement process will be complicated.",
]

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class AI_TASK(TypedDict):
    question: str
    answer: str
    explanation: str


class State(TypedDict):
    intent: str
    drift: AI_TASK
    domain: str
    prompt: str
    drift_value: int
    drift_threshold: int


# Nodes
def setup(state: State):
    state["drift_value"] = 0
    return state


# Nodes
def drift_detection(state: State):
    examples = fewshot.get_examples()
    prefix = fewshot.get_prefix()
    suffix = fewshot.get_suffix()
    example_template, example_variables = fewshot.get_example_template()
    fewShot = FewShotUtility(
        examples=examples,
        prefix=prefix,
        suffix=suffix,
        input_variables=["question"],
        example_template=example_template,
        example_variables=example_variables,
    )
    for prompt in prompts:
        question = (
            "For the prompt: "
            + prompt
            + " Consider a binary text classification problem. which class does the prompt belong? Strictly choose one of the two classes. The options are: (1) "
            + state["domain"]
            + " or (2) other"
        )
        prompt = fewShot.get_prompt(question)
        msg = fewShot.user_drift_detection(prompt)
        if "other" == msg["answer"]:
            state["drift_value"] = state["drift_value"] + 1
        else:
            state["drift_value"] = state["drift_value"]

        print(
            msg,
            "\nNumber of drifts: " + str(state["drift_value"]),
        )
    return {"drift_value": state["drift_value"]}


def incident_reporting(state: State):
    if state["drift_value"] > state["drift_threshold"]:
        incident_report = (
            "Alert: Potential drift in prompts identified.\nDrift Threshold: "
            + str(state["drift_threshold"])
            + "\nNumber of identified drifts: "
            + str(state["drift_value"])
            + "\n"
        )
        print("\n---------\n")
        print(incident_report)
        print("\n---------\n")

    else:
        incident_report = "None"
        print("No drift detected\n")
    return {"incident_report": incident_report}


# Build workflow
workflow = StateGraph(State)

# Add nodes
workflow.add_node("setup", setup)
workflow.add_node("drift_detection", drift_detection)
workflow.add_node("incident_reporting", incident_reporting)


# Add edges to connect nodes
workflow.add_edge(START, "setup")
workflow.add_edge("setup", "drift_detection")
workflow.add_edge("drift_detection", "incident_reporting")

workflow.add_edge("incident_reporting", END)

# Compile
chain = workflow.compile()

# Show workflow
display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
# Invoke
print("Check drift in prompts:")
print("\n--- --- ---\n")
state = chain.invoke(
    {"drift_threshold": 2, "intent": usecase, "domain": "customer claims"}
)
print("\n--- --- ---\n")

Check drift in prompts:

--- --- ---

{'answer': 'customer claims', 'explanation': 'The prompt belongs to the class of customer claims because it involves a specific inquiry from a customer regarding a return of a defective product, which is a common scenario in customer service.', 'question': 'For the prompt:Create a personalized response for a customer who inquired about returning a defective product. The customer'} 
Number of drifts: 0
{'answer': 'customer claims', 'explanation': 'The answer was generated based on the classification of the prompt as a customer-related claim, indicating that it belongs to the category of customer claims.', 'question': 'For the prompt: Generate a summary of claims for a customer who inquired about the '} 
Number of drifts: 0
{'answer': 'other', 'explanation': 'The prompt does not contain any specific keywords related to healthcare insurance, waste management, or customer claims, which are the primary focus areas of these industries. Therefore, it is c